In [60]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [43]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [19]:
fake_B = torch.zeros((3,4,5))
fake_B[0][0][1] = 1
fake_B[0][1][0] = 1
fake_B[0][2][3] = 1
fake_B

tensor([[[0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

In [57]:

a  = (fake_B[:, :,0] != 1).long()

In [62]:
model = torch.load('./model/D_B.pt').eval()

In [ ]:
y,y_attn = en(['a certain number of workers in certain operating sites, on which this type of directive a due due to practical circumstances circumstances, only a difficult difficult to to use.....', 'I myself from Schottland, a region with one of the largest fishing fishing fleets in the EU EU, will again again on the problem of fisheries and the definition of the definition of the working time..................'])

In [ ]:
label,label_attn = en(['There would remain though some workers in some working places where practical issues do not readily lend themselves to the implementation of this type of directive.', 'Coming from Scotland, with one of the largest EU fishing fleets, I would like once again to focus on the problem of fisheries and the defining of working time.'])

In [ ]:
model(y.cuda(),y_attn.cuda())

tensor([[0.9445],
        [0.5153]], device='cuda:0', grad_fn=<ReluBackward0>)

In [ ]:
model(label.cuda(),label_attn.cuda())

tensor([[1.0948],
        [0.6897]], device='cuda:0', grad_fn=<ReluBackward0>)

In [107]:
#real sentences
x = [[10180,   351, 19810,     7,    24,     6,    38,     8,  8160,  8675,
            13,     8,  1611, 15375,     6,    62,   225,     6,    11,  5071,
           398,     6,  2173,   981, 12361,   485,    11, 13567,    16,     8,
          1028,  5808,     7,  1194,    11,  6572,    53,    13,    48,   540,
             6, 10321,    69, 12123,     7,    11,   811,     7,  9098,    12,
             3,  9582,   125,    33,   801,    38,    96, 12097,    21,   540,
           121, 15600,    16,     8,  5334,    18, 14259,   433,     5,     1,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [ 3021,     6,    62,    66,   396,   557,   217,  4248,  4505,     7,
            13,   540,   271,  1869,    30,  1195,     3,  2544,  6353,    56,
          6539,    36, 19363,    44,     8,   456,    13,     8,  2486,  1059,
             5,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]
       ]
             

x_ = torch.Tensor(x).long().cuda()
x_ = x_[:,:100]
x_attn = (x_>0.5).long()
print(x_.shape)
print(x_attn.shape)


torch.Size([2, 100])
torch.Size([2, 100])


In [108]:

x_emb = model.embedding(x_)#
print('x_emb.shape',x_emb.shape)
distr = model.encoder(inputs_embeds=x_emb).last_hidden_state#(bs,sentence length,512)
print('distr.shape',distr.shape)
x_attn= x_attn.unsqueeze(-1)
print('x_attn.shape',x_attn.shape)
distr = torch.mul(distr,x_attn)#previously
print(distr)
print(x_attn)
print(torch.sum(x_attn,1))
distr = torch.sum(distr,1)/torch.sum(x_attn,1)#(bs,512)
print(distr)




# a_pred_dis = model(x_,x_attn)
# a_pred_dis

x_emb.shape torch.Size([2, 100, 512])
distr.shape torch.Size([2, 100, 512])
x_attn.shape torch.Size([2, 100, 1])
tensor([[[ 0.7266,  0.2011,  0.0084,  ..., -0.0780,  0.1437,  0.3328],
         [ 0.5346, -0.0586,  0.2039,  ..., -0.1032,  0.1125,  0.1686],
         [ 0.0174,  0.0663,  0.3278,  ...,  0.0865,  0.0615,  0.1850],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.0260,  0.0907,  0.2656,  ...,  0.0650,  0.0353,  0.2748],
         [ 0.4851,  0.1319,  0.0138,  ...,  0.1788,  0.1812, -0.2455],
         [ 0.1792,  0.3723,  0.1188,  ...,  0.1677,  0.1189,  0.0696],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
   

## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier